This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/aloha).

## Aloha Demo

In this notebook we will walk through a simple pipeline deployment to inference on a model. For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model.
* Create a pipeline that can ingest our submitted data, submit it to the model, and export the results
* Run a sample inference through our pipeline by loading a file
* Run a sample inference through our pipeline's URL and store the results in a file.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [20]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)

# to display dataframe tables
from IPython.display import display

In [ ]:
# Client connection from local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR SUFFIX"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [22]:
import os
arrowEnabled=True
os.environ["ARROW_ENABLED"]=f"{arrowEnabled}"


## Create the Workspace

We will create a workspace to work in and call it the "alohaworkspace", then set it as current workspace environment.  We'll also create our pipeline in advance as `alohapipeline`.  The model name and the model file will be specified for use in later steps.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [23]:
import string
import random

# make a random 4 character prefix
prefix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
workspace_name = f'{prefix}alohaworkspace'
pipeline_name = f'{prefix}alohapipeline'
model_name = f'{prefix}alohamodel'
model_file_name = './alohacnnlstm.zip'

In [24]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [25]:
wl.list_workspaces()

Name,Created At,Users,Models,Pipelines
john.hummel@wallaroo.ai - Default Workspace,2023-02-16 18:24:23,['john.hummel@wallaroo.ai'],0,0
opuvccfraudworkspace,2023-02-16 18:27:16,['john.hummel@wallaroo.ai'],1,1
dpptimdbworkspace,2023-02-16 18:33:36,['john.hummel@wallaroo.ai'],2,1
puclhotswapworkspace,2023-02-16 18:52:48,['john.hummel@wallaroo.ai'],2,1


In [26]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

aloha_pipeline = get_pipeline(pipeline_name)
aloha_pipeline

name,ufonalohapipeline
created,2023-02-16 19:10:17.367944+00:00
last_updated,2023-02-16 19:10:17.367944+00:00
deployed,(none)
tags,
versions,fc3519f0-96e8-4d55-96b5-439f19b30b5d
steps,


We can verify the workspace is created the current default workspace with the `get_current_workspace()` command.

In [27]:
wl.get_current_workspace()

{'name': 'ufonalohaworkspace', 'id': 8, 'archived': False, 'created_by': 'bada4bc7-5e14-4bd9-8ef0-596a58f3d80d', 'created_at': '2023-02-16T19:10:16.629627+00:00', 'models': [], 'pipelines': [{'name': 'ufonalohapipeline', 'create_time': datetime.datetime(2023, 2, 16, 19, 10, 17, 367944, tzinfo=tzutc()), 'definition': '[]'}]}

# Upload the Models

Now we will upload our models.  Note that for this example we are applying the model from a .ZIP file.  The Aloha model is a [protobuf](https://developers.google.com/protocol-buffers) file that has been defined for evaluating web pages, and we will configure it to use data in the `tensorflow` format.

In [28]:
model = wl.upload_model(model_name, model_file_name).configure("tensorflow")

## Deploy a model

Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment.

To do this, we'll create our pipeline that can ingest the data, pass the data to our Aloha model, and give us a final output.  We'll call our pipeline `aloha-test-demo`, then deploy it so it's ready to receive data.  The deployment process usually takes about 45 seconds.

* **Note**:  If you receive an error that the pipeline could not be deployed because there are not enough resources, undeploy any other pipelines and deploy this one again.  This command can quickly undeploy all pipelines to regain resources.  We recommend **not** running this command in a production environment since it will cancel any running pipelines:

```python
for p in wl.list_pipelines(): p.undeploy()
```

In [29]:
aloha_pipeline.add_model_step(model)

name,ufonalohapipeline
created,2023-02-16 19:10:17.367944+00:00
last_updated,2023-02-16 19:10:17.367944+00:00
deployed,(none)
tags,
versions,fc3519f0-96e8-4d55-96b5-439f19b30b5d
steps,


In [30]:
aloha_pipeline.deploy()

name,ufonalohapipeline
created,2023-02-16 19:10:17.367944+00:00
last_updated,2023-02-16 19:10:22.844172+00:00
deployed,True
tags,
versions,"0feb9059-bf2b-42b0-9957-423ed8b60222, fc3519f0-96e8-4d55-96b5-439f19b30b5d"
steps,ufonalohamodel


We can verify that the pipeline is running and list what models are associated with it.

In [31]:
aloha_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.12.17',
   'name': 'engine-ddccf6d4c-xj7ns',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'ufonalohapipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ufonalohamodel',
      'version': 'cfdab747-0e03-4fb7-8f65-f09e774a9566',
      'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.17.9',
   'name': 'engine-lb-ddd995646-7r4dm',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

## Interferences

### Infer 1 row

Now that the pipeline is deployed and our Aloha model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single encoded URL into the inference engine and print the results back out.

The result should tell us that the tokenized URL is legitimate (0) or fraud (1).  This sample data should return close to 0.

In [32]:
if arrowEnabled is True:
    result = aloha_pipeline.infer_from_file('./data/data_1.df.json')
else:
    result = aloha_pipeline.infer_from_file("./data/data_1.json")
display(result)

,time,in.text_input,out.cryptolocker,out.kraken,out.ramnit,out.suppobox,out.banjori,out.ramdo,out.matsnu,out.simda,out.corebot,out.pykspa,out.dircrypt,out.qakbot,out.locky,out.main,out.gozi,check_failures
0,2023-02-16 19:10:35.739,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.012099565],[0.0003197726],[0.0009985751],[1.3889951e-27],[0.0015195857],[0.006236233],[0.010341615],[1.793378e-26],[0.9829148],[0.008038961],[4.7591297e-05],[0.016155062],[0.011029283],[0.997564],[2.0289372e-05],0


### Batch Inference

Now that our smoke test is successful, let's really give it some data.  We have two inference files we can use:

* `data_1k.json`:  Contains 10,000 inferences
* `data_25k.json`: Contains 25,000 inferences

We'll pipe the `data_25k.json` file through the `aloha_pipeline` deployment URL, and place the results in a file named `response.txt`.  We'll also display the time this takes.  Note that for larger batches of 50,000 inferences or more can be difficult to view in Jupyter Hub because of its size.

* **IMPORTANT NOTE**:  The `_deployment._url()` method will return an **internal** URL when using Python commands from within the Wallaroo instance - for example, the Wallaroo JupyterHub service.  When connecting via an external connection, `_deployment._url()` returns an **external** URL.  External URL connections requires [the authentication be included in the HTTP request](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/), and that [Model Endpoints Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-model-endpoints-guide/) external endpoints are enabled in the Wallaroo configuration options.

In [33]:
inference_url = aloha_pipeline._deployment._url()
inference_url

'https://doc-test.api.wallaroocommunity.ninja/v1/api/pipelines/infer/ufonalohapipeline-5'

In [34]:
connection =wl.mlops().__dict__
token = connection['token']
token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJDUGdjeVNiNWNZaENlckRPTEFYNDU0LXkxYjhjQlgwbTd5ajhSVFNnQ3hrIn0.eyJleHAiOjE2NzY1NzQ2NjQsImlhdCI6MTY3NjU3NDYwNCwiYXV0aF90aW1lIjoxNjc2NTcxODYwLCJqdGkiOiIwOWZjZTY0Yi02MDFiLTQ1NGMtOTYzYy05MGQxNjZlYmNjYzAiLCJpc3MiOiJodHRwczovL2RvYy10ZXN0LmtleWNsb2FrLndhbGxhcm9vY29tbXVuaXR5Lm5pbmphL2F1dGgvcmVhbG1zL21hc3RlciIsImF1ZCI6WyJtYXN0ZXItcmVhbG0iLCJhY2NvdW50Il0sInN1YiI6ImJhZGE0YmM3LTVlMTQtNGJkOS04ZWYwLTU5NmE1OGYzZDgwZCIsInR5cCI6IkJlYXJlciIsImF6cCI6InNkay1jbGllbnQiLCJzZXNzaW9uX3N0YXRlIjoiMWQ3YmJmYzktZTNiOC00YzAzLWI2NmItZTdkZjY2ZDhlODU5IiwiYWNyIjoiMCIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJkZWZhdWx0LXJvbGVzLW1hc3RlciIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJtYXN0ZXItcmVhbG0iOnsicm9sZXMiOlsibWFuYWdlLXVzZXJzIiwidmlldy11c2VycyIsInF1ZXJ5LWdyb3VwcyIsInF1ZXJ5LXVzZXJzIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6InByb2ZpbGUgZW1haWwiLCJzaWQiOiIxZDdiYmZjOS1

In [35]:
if arrowEnabled is True:
    dataFile="./data/data_25k.df.json"
    contentType="application/json; format=pandas-records"
else:
    dataFile="./data/data_25k.json"
    contentType="application/json"

In [36]:
!curl -X POST {inference_url} -H "Authorization: Bearer {token}" -H "Content-Type:{contentType}" --data @{dataFile} > curl_response.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.4M  100 16.3M  100 18.0M  1507k  1659k  0:00:11  0:00:11 --:--:-- 4650k


## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.  Note that if the deployment variable is unchanged aloha_pipeline.deploy() will restart the inference engine in the same configuration as before.

In [37]:
aloha_pipeline.undeploy()

name,ufonalohapipeline
created,2023-02-16 19:10:17.367944+00:00
last_updated,2023-02-16 19:10:22.844172+00:00
deployed,False
tags,
versions,"0feb9059-bf2b-42b0-9957-423ed8b60222, fc3519f0-96e8-4d55-96b5-439f19b30b5d"
steps,ufonalohamodel
